In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
import json
import csv
import math

In [11]:

def buildFirstNodes(message):
    charSet = {}
    totalCharacterCount = 0
    
    for char in message:
        totalCharacterCount+=1
        if char in charSet:
            charSet[char] += 1
        else:
            charSet[char] = 1
    
    for char in charSet:
        charDataObj = {}
        charDataObj["symbol"] = char
        charDataObj["count"] = charSet[char]
        charDataObj["frequency"] = charSet[char]/totalCharacterCount 
        charDataObj["code"] = ""
        charDataObj["children"] = []
        leafNodesArray.append(charDataObj)
    
    leafNodesArray.sort(key=lambda i : i["frequency"])

In [12]:
def mergeNodes():#array of nodes is passed
    node0 = leafNodesArray.pop(0)
    node1 = leafNodesArray.pop(0)

    newNode = {
        "frequency" : node0["frequency"]+node1["frequency"],
        "children": [node0, node1],
        "code": ""
    }
    leafNodesArray.append(newNode)
    leafNodesArray.sort(key=lambda i : i["frequency"])
    while(len(leafNodesArray)>=2):
      mergeNodes()
 

In [13]:
def assignCodes(node, children, code):
    children[0]["code"] = code+"0"
    children[1]["code"] = code+"1"
    if len(children[0]["children"])>1:
        assignCodes(children[0],children[0]["children"], children[0]["code"])
    else:
      charDataObj = {"count":children[0]["count"], "frequency": children[0]["frequency"],"code": children[0]["code"]}
      codes[children[0]["symbol"]] = charDataObj

    if len(children[1]["children"])>1:
        assignCodes(children[1],children[1]["children"], children[1]["code"])
    else:
      charDataObj = {"count":children[1]["count"], "frequency": children[1]["frequency"],"code": children[1]["code"]}
      codes[children[1]["symbol"]] = charDataObj


In [14]:
def encode(message):
    buildFirstNodes(message)
    mergeNodes()
    tree = leafNodesArray[0]
    assignCodes(tree, tree["children"], tree["code"])

In [15]:
codes = {}
leafNodesArray = []


# open text file in read mode
text_file = open("codingDataExample.txt", "r")

# read whole file to a string
message = text_file.read()

# close file
text_file.close()


# Provide different values for message and test your program
encoded_message = encode(message)
# Serializing json
json_tree = json.dumps(leafNodesArray[0], indent=4)
 
# Writing to sample.json
with open("CodingTree.json", "w") as outfile:
    outfile.write(json_tree)


	
# field names
fields = ['Symbol', 'Count', 'Frequency', 'Code']
	
# name of csv file
filename = "Codes.csv"
	
# writing to csv file
with open(filename, 'w') as csvfile:
	# creating a csv dict writer object
	csvwriter = csv.writer(csvfile) 
	# writing headers (field names)
	csvwriter.writerow(fields)
	# writing data rows
	csvwriter.writerows([[key,codes[key]["count"], codes[key]["frequency"], codes[key]["code"]] for key in codes])


lenOfEncodedMessage = 0
entropy = 0
averageCodeLength = 0
for key in codes:
		lenOfEncodedMessage += codes[key]["count"] * len(codes[key]["code"])
		entropy -= codes[key]["frequency"] * math.log(codes[key]["frequency"], 2)
		averageCodeLength += codes[key]["frequency"] * len(codes[key]["code"])


print("Assuming that 8 bits are used to encode evey character in the uncompressed file.")
print("Cmpression ratio = {:.5}".format((len(message)*8)/lenOfEncodedMessage))
print("Entropy = {:.5}".format(entropy))
print("Average Code Length = {:.5}".format(averageCodeLength))
print("Efficiency = {:.5}".format(entropy/averageCodeLength))


textOutputFile = open("OutputText.txt","w")

print("Assuming that 8 bits are used to encode evey character in the uncompressed file.", file=textOutputFile)
print("Cmpression ratio = {:.5}".format((len(message)*8)/lenOfEncodedMessage), file=textOutputFile)
print("Entropy = {:.5}".format(entropy), file=textOutputFile)
print("Average Code Length = {:.5}".format(averageCodeLength), file=textOutputFile)
print("Efficiency = {:.5}".format(entropy/averageCodeLength), file=textOutputFile)

textOutputFile.close()

Assuming that 8 bits are used to encode evey character in the uncompressed file.
Cmpression ratio = 2.1078
Entropy = 3.7591
Average Code Length = 3.7955
Efficiency = 0.99043
